**Nhập các thư viện**

In [1]:
import pandas as pd
import numpy as np
import os
import re

RAW_DATA_PATH = r"../data/raw"
PROCESSED_DATA_PATH = r"../data/processed"

### Xử lý các file time series từ cumulative sang daily
#### Mục tiêu:
- Nhóm các dòng thuộc cùng 1 quốc gia lại với nhau
- Xử lý dữ liệu từ thống kê tích lũy (cumulative) sang thống kê hằng ngày (daily) để có cái nhìn tổng quan hơn về số ca nhiễm, tử vong, hồi phục mới sau mỗi ngày

**Mở các file .csv liên quan đến time series và xuất ra dataframe**

In [2]:
global_time_series_files = [
    'time_series_covid_19_confirmed.csv',
    'time_series_covid_19_deaths.csv',
    'time_series_covid_19_recovered.csv'
]

In [3]:
global_time_series_data = {}
for file in global_time_series_files:
    df = pd.read_csv(os.path.join(RAW_DATA_PATH, file))
    key = file.replace("time_series_covid_19_", "").replace(".csv", "")
    global_time_series_data[key] = df

    print(f"{key}: {df.shape}")

confirmed: (276, 498)
deaths: (276, 498)
recovered: (261, 498)


In [4]:
confirmed_df = global_time_series_data.get("confirmed")
deaths_df = global_time_series_data.get("deaths")
recovered_df = global_time_series_data.get("recovered")

Theo file data_collection, có vài dòng chứa NaN, ta kiểm tra thử nội dung trong dòng đó như thế nào

In [5]:
columns_to_exclude = ['Province/State']  # Thay bằng các cột của bạn

df_to_check = confirmed_df.drop(columns=columns_to_exclude)

# Tìm các dòng có NaN trong các cột còn lại
rows_with_nan = confirmed_df[df_to_check.isna().any(axis=1)]
rows_with_nan

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,5/20/21,5/21/21,5/22/21,5/23/21,5/24/21,5/25/21,5/26/21,5/27/21,5/28/21,5/29/21
52,Repatriated Travellers,Canada,NaN,NaN,0,0,0,0,0,0,...,13,13,13,13,13,13,13,13,13,13
88,Unknown,China,NaN,NaN,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Giá trị NaN là do tọa độ ở các điểm không xác định, điều này không quan trọng vì ta sẽ nhóm các dòng từ cùng một quốc gia lại với nhau nên sẽ có tọa độ lấy từ điểm hợp lệ đầu tiên trong mỗi quốc gia

Viết hàm xử lý bộ dữ liệu.


In [6]:
def process_covid_data(df):
    """
    Hàm xử lý chung cho cả 3 loại dữ liệu (confirmed, deaths, recovered)
    Chuyển từ cumulative sang daily cases và giữ lại tọa độ
    """
    # Gom nhóm theo quốc gia và tính tổng
    processed_df = df.groupby('Country/Region').agg({
    'Lat': 'first', #Lấy tọa độ từ giá trị đầu tiên trong mỗi nhóm quốc giâ
    'Long': 'first',
    **{col: 'sum' for col in df.columns if col not in ['Country/Region', 'Province/State', 'Lat', 'Long']}
    })
    processed_df.reset_index(inplace=True)

    # Tách tọa độ và dữ liệu 
    location = processed_df[['Country/Region', 'Lat', 'Long']]
    cumulative_cases = processed_df.drop(columns=['Country/Region', 'Lat', 'Long'])
    
    # Chuyển sang daily cases
    daily_cases = cumulative_cases.copy()
    daily_cases.iloc[:, 1:] = daily_cases.iloc[:, 1:].values - daily_cases.iloc[:, 0:-1].values
    
    # Kết hợp lại với tọa độ
    result = pd.concat([location, daily_cases], axis=1)
    
    return result

Xử lý chung cho cả 3 file dựa vào hàm xử lý

In [7]:
# Dictionary để lưu kết quả
daily_data = {}

# Xử lý từng file
for key in global_time_series_data.keys():
    df = global_time_series_data.get(key)
    
    # Tạo key cho dictionary (confirmed, deaths, recovered)
    # Xử lý dữ liệu
    daily_df = process_covid_data(df)
    
    # Lưu vào dictionary
    daily_data[key] = daily_df
    
    print(f"Đã xử lý xong {key}: {daily_df.shape}")

# Truy cập các DataFrame đã xử lý
daily_confirmed = daily_data.get("confirmed")
daily_deaths = daily_data.get("deaths")
daily_recovered = daily_data.get("recovered")

# Kiểm tra kết quả
print("\nKích thước các DataFrame đã xử lý:")
print(f"Daily confirmed cases: {daily_confirmed.shape}")
print(f"Daily deaths: {daily_deaths.shape}")
print(f"Daily recovered: {daily_recovered.shape}")

Đã xử lý xong confirmed: (193, 497)
Đã xử lý xong deaths: (193, 497)
Đã xử lý xong recovered: (193, 497)

Kích thước các DataFrame đã xử lý:
Daily confirmed cases: (193, 497)
Daily deaths: (193, 497)
Daily recovered: (193, 497)


Kiểm tra lại dữ liệu sau khi xử lý có giá trị âm trong các cases hay không

In [8]:
# Lấy daily_confirmed_dropped làm ví dụ
# Tìm tất cả các ô có giá trị âm trong các cột số
daily_confirmed_dropped = daily_confirmed.drop(columns=['Lat', 'Long'])
negative_values = daily_confirmed_dropped.select_dtypes(include=['number'])[daily_confirmed_dropped.select_dtypes(include=['number']) < 0]

# Tạo danh sách để lưu kết quả
negative_list = []

# Duyệt qua các cột có giá trị âm
for col in negative_values.columns:
    # Lấy các hàng có giá trị âm trong cột này
    neg_rows = negative_values[col].dropna()
    
    # Thêm vào danh sách kết quả
    for idx, value in neg_rows.items():
        country = daily_confirmed_dropped.loc[idx, 'Country/Region']  # Hoặc 'Country' tùy tên cột của bạn
        negative_list.append({
            'Country': country,
            'Date': col.replace('Daily_', '') if col.startswith('Daily_') else col,
            'Negative_Value': value
        })

# Tạo DataFrame từ danh sách
result = pd.DataFrame(negative_list)

result

,Country,Date,Negative_Value
0,Israel,3/11/20,-28.0
1,Guyana,3/24/20,-15.0
2,France,4/4/20,-17074.0
3,France,4/7/20,-3491.0
4,Uruguay,4/12/20,-21.0
...,...,...,...
61,Antigua and Barbuda,5/9/21,-1.0
62,Mozambique,5/11/21,-16.0
63,New Zealand,5/11/21,-1.0
64,United Kingdom,5/18/21,-2362.0


**Nguyên nhân**: Có thể do bộ phận thống kê loại bỏ các ca nhiễm dương tính giả, hoặc tiêu chuẩn đánh giá dương tính thay đổi trong quá trình thống kê. \
**Hướng giải quyết**: Chuyển các giá trị âm về 0.

In [9]:
cols_to_exclude = ['Country/Region', 'Lat', 'Long']  # Thay bằng các cột bạn muốn giữ nguyên
for name, df in daily_data.items():
    # Lọc các cột cần thay đổi (không nằm trong cols_to_exclude)
    cols_to_update = [col for col in df.columns if col not in cols_to_exclude]
    
    # Thay thế giá trị âm bằng 0 trong các cột được chọn
    df[cols_to_update] = df[cols_to_update].map(lambda x: x if x >= 0 else 0)

Lưu file lại

In [10]:
output_dir = os.path.join(PROCESSED_DATA_PATH, 'daily_data')
os.makedirs(output_dir, exist_ok=True)

# Xuất từng DataFrame sang file CSV
for data_type, df in daily_data.items():
    # Tạo tên file
    filename = f"daily_{data_type}_cases.csv"
    filepath = os.path.join(output_dir, filename)
    
    # Xuất file CSV
    df.to_csv(filepath, index=False)
    print(f"Đã xuất file {filename} thành công")

print(f"\nTất cả file đã được lưu tại: {output_dir}")

Đã xuất file daily_confirmed_cases.csv thành công
Đã xuất file daily_deaths_cases.csv thành công
Đã xuất file daily_recovered_cases.csv thành công

Tất cả file đã được lưu tại: ../data/processed\daily_data
